In [0]:
!pip install tensorflow==1.14.0

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2

from PIL import Image

In [0]:
from google.colab import drive #STU NAPERVILLE ACCOUNTS DON'T WORK - USE YOUR PERSONAL GOOGLE ACCOUNTS
drive.mount('/content/drive')

In [0]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255, brightness_range=(-1, 1))
train_generator = train_datagen.flow_from_directory(
        "/content/drive/My Drive/ROADREADER DATASETS/training",
        target_size=(300, 400),
        batch_size=32, 
        color_mode="rgb",
        class_mode="sparse",
        shuffle=True)

test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255, brightness_range=(-1, 1))
test_generator = test_datagen.flow_from_directory(
        "/content/drive/My Drive/ROADREADER DATASETS/tests",
        target_size=(300, 400),
        batch_size=32, 
        color_mode="rgb",
        class_mode="sparse",
        shuffle=True)

Found 1196 images belonging to 2 classes.
Found 204 images belonging to 2 classes.


In [0]:
model = keras.Sequential([
  layers.Conv2D(14, 3, activation='relu', input_shape=(300,400,3)),
  layers.MaxPooling2D((2,2), padding='same'),
  layers.Conv2D(28, 3, activation='relu'),
  layers.MaxPooling2D((2,2), padding='same'),
  layers.Conv2D(42, 3, activation='relu'),
  layers.MaxPooling2D((2,2), padding='same'),
  layers.Conv2D(84, 3, activation='relu'),
  layers.MaxPooling2D((2,2), padding='same'),
  layers.Conv2D(84, 3, activation='relu'),
  layers.MaxPooling2D((2,2), padding='same'),
  layers.Flatten(),
  layers.Dense(200, activation='relu'),
  layers.Dense(2, activation='softmax'), #classification layer; 0 for none 1 for stop
])
model.summary()

In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [0]:
#model.fit(x=train_images,y=train_labels, validation_data=(test_images, test_labels), epochs=10, batch_size=128)

model.fit_generator(
        train_generator,
        steps_per_epoch=64,
        epochs=20,
        validation_data=test_generator,
        validation_steps=10,
        workers=6,
        use_multiprocessing=True
        #max_queue_size=10
        )

'''
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)
                    '''

model.save_weights('/content/model.h5')

Epoch 1/20
64/64 [==============================] - 163s 3s/step - loss: 0.7148 - acc: 0.6290 - val_loss: 0.5236 - val_acc: 1.0000
Epoch 2/20
64/64 [==============================] - 164s 3s/step - loss: 0.4828 - acc: 0.6992 - val_loss: 0.6197 - val_acc: 0.9000
Epoch 3/20
64/64 [==============================] - 164s 3s/step - loss: 0.4560 - acc: 0.7061 - val_loss: 0.1187 - val_acc: 1.0000
Epoch 4/20
64/64 [==============================] - 158s 2s/step - loss: 0.4167 - acc: 0.7209 - val_loss: 0.6323 - val_acc: 1.0000
Epoch 5/20
64/64 [==============================] - 163s 3s/step - loss: 0.4672 - acc: 0.6942 - val_loss: 0.9110 - val_acc: 0.0000e+00
Epoch 6/20
64/64 [==============================] - 164s 3s/step - loss: 0.4376 - acc: 0.7125 - val_loss: 0.7440 - val_acc: 0.0000e+00
Epoch 7/20
64/64 [==============================] - 156s 2s/step - loss: 0.4295 - acc: 0.7196 - val_loss: 0.8899 - val_acc: 0.0000e+00
Epoch 8/20
64/64 [==============================] - 164s 3s/step - loss